<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=3cd27f82c3d6b6b6539be796d27518e55d8c704bbe787bc1213c6247e6469307
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-26 11:47:36
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: 1.18 L (0.83%)
Current PnL: -22.45 L (-14.71%)
CY Booked + Current PnL: -8.56 L (-5.61%)
-------------------
Total profit:  1.54 L
Total loss:  -23.99 L
-------------------
Total Booked + Current PnL: 18.42 L (14.66%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.69%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 87.86 L (61.3%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.5%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,2.36,-7.18,7.76,0.02,12573.0,-12541.0,162018.0,147.21,73.0,M-SC,7.91,234.0,-1.00,1.13,40.93,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.65,-13.54,15.77,0.09,13740.0,-13649.0,87128.0,102.13,53.0,M-SC,3.73,253.0,-0.99,0.61,14.47,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,1.99,8.22,10.61,19.69,22015.0,15753.0,207495.0,-2.32,52.0,X-MC,3.39,78.0,0.72,1.45,15.57,XY25,NTT,AC
35,ICICIGI,2252.93,0.43,7.11,12.22,20.19,22816.0,12387.0,186707.0,-16.02,52.0,X-MC,7.46,68.0,0.54,1.30,23.29,X40,ATH,INSURANCE
53,NESTLEIND,1377.00,1.20,12.70,7.71,21.40,23064.0,33720.0,299146.0,4.55,59.0,X-LC,7.80,12.0,1.46,2.09,20.19,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIEMENS,4671.50,4.54,-10.65,40.48,25.51,67306.0,-19825.0,166270.0,1.98,67.0,H-LC,1.90,49.0,-0.29,1.16,22.53,AR,ATH,ELECTRICAL
64,SFL,1287.00,3.96,-39.33,108.73,26.63,172971.0,-103135.0,159083.0,20.57,37.0,M-SC,4.93,239.0,-0.60,1.11,3.96,XY24,NTT,MISC
0,5PAISA,593.00,3.46,-34.30,71.83,12.90,103121.0,-74934.0,143562.0,111.54,60.0,H-SC,13.95,161.0,-0.73,1.00,19.37,OX40N,NTT,FINANCE
81,VALIANTORG,838.00,3.10,-46.14,223.36,74.15,196805.0,-75494.0,88111.0,-258.45,30.0,H-SC,16.30,149.0,-0.38,0.61,12.94,XR,NTT,CHEMICALS
22,DIXON,18931.72,2.90,-4.84,27.76,21.58,37021.0,-6777.0,133362.0,-52.01,38.0,X-MC,6.70,56.0,-0.18,0.93,18.07,X40N,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JCHAC,2282.00,-1.72,-24.81,32.98,-0.01,31694.0,-31703.0,96102.0,19573.91,32.0,M-SC,1.60,233.0,-1.00,0.67,8.82,OX40N,NTT,AC
38,INDIAMART,4810.62,-1.61,-0.24,107.22,106.72,131923.0,-296.0,123040.0,-50.63,37.0,H-SC,3.37,139.0,-0.00,0.86,23.08,AR,ATH,MISC
56,QUESS,424.00,-1.47,-27.46,96.91,42.84,45700.0,-17849.0,47157.0,-52.62,42.0,X-SC,37.46,83.0,-0.39,0.33,2.61,XY24,NTT,MISC
62,ROUTE,2227.21,-1.37,-49.98,237.97,69.05,158390.0,-66507.0,66559.0,-59.60,36.0,H-SC,23.70,140.0,-0.42,0.46,2.65,SR,ATH,IT
69,STARHEALTH,761.00,-1.29,-5.21,55.35,47.26,147496.0,-14645.0,266478.0,29.34,48.0,H-SC,15.49,171.0,-0.10,1.86,42.77,XY24,NTT,INSURANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-1.61,-0.24,107.22,106.72,131923.0,-296.0,123040.0,-50.63,37.0,H-SC,3.37,139.0,-0.00,0.86,23.08,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,1.18,-0.93,36.90,35.63,76615.0,-1944.0,207630.0,-14.97,42.0,H-MC,4.17,119.0,-0.03,1.45,15.59,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.11,-4.47,117.64,107.91,165925.0,-6599.0,141045.0,-22.17,36.0,M-SC,10.48,216.0,-0.04,0.98,0.86,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,2.36,-7.18,7.76,0.02,12573.0,-12541.0,162018.0,147.21,73.0,M-SC,7.91,234.0,-1.00,1.13,40.93,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.65,-13.54,15.77,0.09,13740.0,-13649.0,87128.0,102.13,53.0,M-SC,3.73,253.0,-0.99,0.61,14.47,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.04,-22.93,47.23,13.47,98161.0,-61830.0,207837.0,-68.31,26.0,H-SC,1.29,158.0,-0.63,1.45,5.68,XY24,NTT,PAINTS
67,SIS,528.00,0.74,-23.30,58.11,21.26,49485.0,-25875.0,85157.0,2010.36,50.0,H-SC,3.87,166.0,-0.52,0.59,15.06,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,0.78,1.53,69.66,72.27,119848.0,2599.0,172047.0,-12.89,57.0,M-LC,4.45,99.0,0.02,1.2,7.8,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.78,1.53,69.66,72.27,119848.0,2599.0,172047.0,-12.89,57.0,M-LC,4.45,99.0,0.02,1.20,7.80,XR,NTT,IT
38,INDIAMART,4810.62,-1.61,-0.24,107.22,106.72,131923.0,-296.0,123040.0,-50.63,37.0,H-SC,3.37,139.0,-0.00,0.86,23.08,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,1.18,-0.93,36.90,35.63,76615.0,-1944.0,207630.0,-14.97,42.0,H-MC,4.17,119.0,-0.03,1.45,15.59,AR,ATH,PHARMA
25,FINCABLES,1641.55,0.11,-4.47,117.64,107.91,165925.0,-6599.0,141045.0,-22.17,36.0,M-SC,10.48,216.0,-0.04,0.98,0.86,OX40N,ATH,CABLES
84,WHIRLPOOL,2270.00,-0.08,-4.29,90.07,81.92,98965.0,-4922.0,109876.0,-43.90,35.0,M-SC,3.80,236.0,-0.05,0.77,30.08,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.60,-35.47,109.54,35.20,90946.0,-45645.0,83025.0,8.88,24.0,X-SC,13.04,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
76,TMPV,600.00,1.01,-15.64,68.54,42.18,159822.0,-43230.0,233180.0,-24.30,25.0,X-LC,1.10,3.0,-0.27,1.63,1.01,XY24,NTT,AUTO
46,JSWINFRA,342.00,0.90,-9.13,26.81,15.23,48807.0,-18299.0,182048.0,-22.32,28.0,X-MC,6.78,66.0,-0.37,1.27,20.17,X40N,NTT,REALTY
55,PGHH,17907.65,-0.48,-4.68,40.32,33.76,77185.0,-9390.0,191430.0,-32.78,32.0,X-MC,3.55,57.0,-0.12,1.34,0.28,X40,ATH,FMCG
9,BAJAJHFL,181.50,0.42,-13.79,72.20,48.45,124802.0,-27650.0,172856.0,-20.09,32.0,X-MC,7.07,64.0,-0.22,1.21,0.42,X40N,ATH,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,0.45,-1.63,12.27,10.44,24156.0,-3267.0,196871.0,-41.15,43.0,X-LC,0.89,5.0,-0.14,1.37,3.84,X40,NTT,FMCG
76,TMPV,600.0,1.01,-15.64,68.54,42.18,159822.0,-43230.0,233180.0,-24.30,25.0,X-LC,1.10,3.0,-0.27,1.63,1.01,XY24,NTT,AUTO
3,ACC,3906.0,-0.30,-21.69,109.61,64.15,204258.0,-51601.0,186350.0,-54.13,52.0,X-SC,1.63,82.0,-0.25,1.30,4.53,XY24,BTT,CEMENT
36,ICICIPRULI,790.0,1.29,2.81,27.39,30.97,49769.0,4966.0,181704.0,-21.38,56.0,X-MC,1.69,75.0,0.10,1.27,15.72,X40,ATH,INSURANCE
66,SIEMENS,4671.5,4.54,-10.65,40.48,25.51,67306.0,-19825.0,166270.0,1.98,67.0,H-LC,1.90,49.0,-0.29,1.16,22.53,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.60,-35.47,109.54,35.20,90946.0,-45645.0,83025.0,8.88,24.0,X-SC,13.04,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
55,PGHH,17907.65,-0.48,-4.68,40.32,33.76,77185.0,-9390.0,191430.0,-32.78,32.0,X-MC,3.55,57.0,-0.12,1.34,0.28,X40,ATH,FMCG
9,BAJAJHFL,181.50,0.42,-13.79,72.20,48.45,124802.0,-27650.0,172856.0,-20.09,32.0,X-MC,7.07,64.0,-0.22,1.21,0.42,X40N,ATH,FINANCE
54,PAGEIND,51605.07,0.95,-4.97,32.66,26.07,50819.0,-8140.0,155600.0,-30.06,38.0,X-MC,9.19,55.0,-0.16,1.09,0.95,X40,ATH,APPARELS
76,TMPV,600.00,1.01,-15.64,68.54,42.18,159822.0,-43230.0,233180.0,-24.30,25.0,X-LC,1.10,3.0,-0.27,1.63,1.01,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-1.47,-27.46,96.91,42.84,45700.0,-17849.0,47157.0,-52.62,42.0,X-SC,37.46,83.0,-0.39,0.33,2.61,XY24,NTT,MISC
60,RELAXO,1176.00,1.84,-47.22,188.59,52.31,144479.0,-68550.0,76610.0,-43.86,34.0,X-SC,4.14,91.0,-0.47,0.53,2.55,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.60,-35.47,109.54,35.20,90946.0,-45645.0,83025.0,8.88,24.0,X-SC,13.04,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,2.90,-4.84,27.76,21.58,37021.0,-6777.0,133362.0,-52.01,38.0,X-MC,6.70,56.0,-0.18,0.93,18.07,X40N,ATH,IT
52,MEDANTA,1486.00,0.98,5.36,17.77,24.08,23768.0,6801.0,133751.0,-3.24,50.0,X-SC,5.18,89.0,0.29,0.93,26.02,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,1.39,-14.07,38.80,19.28,115350.0,-48662.0,297294.0,-25.24,63.0,X-LC,7.35,1.0,-0.42,2.07,9.90,X40,ATH,IT
41,INFY,2275.00,0.95,5.19,47.23,54.87,155447.0,16240.0,329128.0,-18.16,59.0,X-LC,2.87,2.0,0.10,2.30,13.90,X40,BTT,IT
76,TMPV,600.00,1.01,-15.64,68.54,42.18,159822.0,-43230.0,233180.0,-24.30,25.0,X-LC,1.10,3.0,-0.27,1.63,1.01,XY24,NTT,AUTO
82,VBL,671.64,0.66,-9.29,49.57,35.67,142015.0,-29348.0,286494.0,-20.14,43.0,X-LC,5.95,4.0,-0.21,2.00,3.27,X40N,ATH,FMCG
43,ITC,452.00,0.45,-1.63,12.27,10.44,24156.0,-3267.0,196871.0,-41.15,43.0,X-LC,0.89,5.0,-0.14,1.37,3.84,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.75,-36.91,110.62,32.89,53885.0,-28494.0,48712.0,-701.92,64.0,L-MC,5.60,259.0,-0.53,0.34,34.15,XR,NTT,BANKS
6,ASIANTILES,137.00,1.41,-11.48,105.24,81.67,87388.0,-10773.0,83037.0,7466.67,56.0,L-SC,19.10,271.0,-0.12,0.58,60.80,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.37,-12.96,60.32,39.54,53112.0,-13108.0,88051.0,-31.96,64.0,M-SC,6.62,220.0,-0.25,0.61,24.16,AR,ATH,AUTO
51,MASFIN,398.61,0.05,-5.76,29.50,22.05,27240.0,-5640.0,92340.0,-18.70,48.0,H-SC,7.38,164.0,-0.21,0.64,34.22,XR,ATH,FINANCE
70,SURYODAY,216.00,1.11,-18.67,51.27,23.03,74673.0,-33425.0,145646.0,57.89,51.0,H-SC,10.16,167.0,-0.45,1.02,44.51,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,2.36,-7.18,7.76,0.02,12573.0,-12541.0,162018.0,147.21,73.0,M-SC,7.91,234.0,-1.00,1.13,40.93,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.44,0.96,19.79,20.95,50304.0,2420.0,254188.0,-0.07,76.0,X-LC,13.97,31.0,0.05,1.77,36.94,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,1.75,-36.91,110.62,32.89,53885.0,-28494.0,48712.0,-701.92,64.0,L-MC,5.60,259.0,-0.53,0.34,34.15,XR,NTT,BANKS
0,5PAISA,593.00,3.46,-34.30,71.83,12.90,103121.0,-74934.0,143562.0,111.54,60.0,H-SC,13.95,161.0,-0.73,1.00,19.37,OX40N,NTT,FINANCE
79,UNITDSPR,1644.00,1.38,6.52,13.37,20.77,33347.0,15273.0,249417.0,-2.78,64.0,X-MC,3.71,62.0,0.46,1.74,13.89,X40N,NTT,BREWERIES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.72
1,25,44.53
2,50,75.48


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.62
MC    29.27
LC    25.09
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.87
X40      21.16
X40N     11.87
XR        9.63
XY25      9.10
AR        9.06
OX40N     7.70
X200      1.84
SR        0.97
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.92
X-MC    22.81
X-LC    19.92
M-SC    12.07
X-SC     8.22
H-MC     4.74
H-LC     3.00
L-SC     1.41
M-MC     1.38
M-LC     1.20
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.72,-4.99,38.94
IT,13.72,-14.87,74.17
FINANCE,9.46,-14.30,62.59
MISC,7.44,-21.81,71.71
ELECTRICAL,5.98,-9.51,49.62
PAINTS,5.95,-8.17,24.24
INSURANCE,4.43,0.43,34.66
PHARMA,3.88,-0.49,32.65
AUTO,3.32,-20.76,72.53


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3041864.0,21
XR,1281059.0,13
AR,1273188.0,10
X40,1005950.0,14
X40N,746919.0,9
OX40N,687286.0,10
XY25,337662.0,6
SR,280205.0,2
MH,72292.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3546546.0,25
M-SC,1330581.0,15
X-MC,1214471.0,16
X-LC,907249.0,11
X-SC,740600.0,8
H-MC,398545.0,3
L-SC,259375.0,3
H-LC,127220.0,2
M-LC,119848.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1215282.0      6
           AR         873733.0      5
           XR         787112.0      7
M-SC       XY24       777266.0      6
X-MC       X40        438273.0      7
X-LC       X40        392988.0      5
X-MC       XY24       355795.0      3
H-SC       OX40N      317922.0      4
X-SC       X40N       292185.0      3
M-SC       OX40N      281837.0      5
H-SC       SR         280205.0      2
X-SC       XY24       273726.0      3
X-MC       X40N       243977.0      4
X-LC       XY24       230287.0      2
           X40N       210757.0      2
H-MC       AR         209037.0      2
           XY24       189508.0      1
X-MC       XY25       176426.0      2
X-SC       X40        174689.0      2
L-SC       XR         171848.0      2
M-SC       XR         148366.0      2
           AR         123112.0      2
M-LC       XR         119848.0      1
L-SC       OX40N       87527.0      1
X-LC       XY25        73217.0      2
H-SC       MH          72292.0      1
H-LC       AR          67306.0      1
           X200        59914.0      1
L-MC       XR          53885.0      1
M-MC       XY25        47941.0      1
L-LC       XY25        40078.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 56.0 seconds
